## Data Loading

In [1]:
import sys
import os
import faiss
import numpy as np
import pandas as pd
from fais_functions import *

sys.path.append(os.path.abspath(".."))

from config import *


In [2]:
df = load_embeddings_and_data(EMBEDEDINGS_FILEPATH_open_ai_api, CSV_FILEPATH_FULLY_PREPROCESSED)
df_UNCHANGED = pd.read_csv(CSV_FILEPATH_UNCHANGED_DATA, index_col=0)
df['description'] = df_UNCHANGED['description']
display(df)

,points,vintage,designation_fe,winery_te,region_1_te,variety_te,province_te,country=Argentina,country=Armenia,country=Australia,...,1527,1528,1529,1530,1531,1532,1533,1534,1535,description
0,87,2013.0,0.000008,30.116707,37.998042,23.197256,28.005311,0,0,0,...,0.019401,0.028897,-0.006622,0.001174,-0.000673,0.039126,-0.018992,0.003420,-0.013108,"Aromas include tropical fruit, broom, brimston..."
1,87,2011.0,0.000015,30.852118,27.113411,24.938141,29.612891,0,0,0,...,-0.019107,0.019660,-0.013772,0.004644,-0.004720,0.022807,0.020652,0.022175,0.004210,"This is ripe and fruity, a wine that is smooth..."
2,87,2013.0,0.288257,24.846995,35.182556,23.028626,36.555187,0,0,0,...,0.009170,0.025412,-0.011315,0.012045,-0.004472,0.030258,-0.012474,0.049850,-0.005594,"Tart and snappy, the flavors of lime flesh and..."
3,87,2013.0,0.000062,26.868550,45.221130,31.681079,33.181793,0,0,0,...,-0.011772,0.013178,-0.033355,0.007023,0.006148,0.018777,0.005071,0.021155,0.002417,"Pineapple rind, lemon pith and orange blossom ..."
4,87,2012.0,0.000008,29.947457,35.096027,47.183510,36.574418,0,0,0,...,-0.010007,0.014762,-0.000711,-0.023990,-0.028092,0.010845,0.013056,0.004186,0.013497,"Much like the regular bottling from 2012, this..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129966,90,2013.0,0.000038,38.014742,27.079127,31.586598,35.498955,0,0,0,...,-0.006448,0.019872,-0.011875,0.013012,0.003279,-0.016738,0.009275,0.036703,0.010354,Notes of honeysuckle and cantaloupe sweeten th...
129967,90,2004.0,0.288257,36.587222,23.731124,47.531364,36.460790,0,0,0,...,0.029040,0.008718,-0.032945,-0.000931,0.015361,-0.008840,0.009545,0.013246,-0.009822,Citation is given as much as a decade of bottl...
129968,90,2013.0,0.000077,40.379782,31.390467,26.853532,30.599440,0,0,0,...,0.005716,0.021218,-0.003018,-0.002218,-0.024211,-0.010474,-0.020153,0.032707,-0.001784,Well-drained gravel soil gives this wine its c...
129969,90,2012.0,0.288257,40.487796,31.003071,22.976374,30.208142,0,0,0,...,0.014323,-0.001026,-0.016832,0.015996,-0.010457,-0.018415,-0.012763,0.036310,-0.016369,"A dry style of Pinot Gris, this is crisp with ..."


## Search From Multiple Entries

In [3]:
def recommend_from_multiple_wines(indices: list, df_vectors: pd.DataFrame, index, k: int = 5):
    selected_vectors = df_vectors.iloc[indices].to_numpy()
    query_vector = np.mean(selected_vectors, axis=0).reshape(1, -1) # Uśredniamy wektory, aby stworzyć jeden wektor zapytania reprezentujący wszystkie wybrane wina
    query_vector = query_vector.astype('float32')
    faiss.normalize_L2(query_vector)
    D, I = index.search(query_vector, k)
    
    return D, I

#### Example

In [4]:
df_nums = df.select_dtypes(include=['number'])
df_nums = df_nums.astype('float32') 
index, df_nums_array = build_faiss_index_from_df_nums(df_nums)
D, I = find_similar_items(index, df_nums_array[1], k=5)
wines_reviews_df, descriptions_list = get_wines_by_indices(I, df)

display(descriptions_list)
display(wines_reviews_df)

print(f"Odległości: {D}")
print(f"Indeksy: {I}")

["This is ripe and fruity, a wine that is smooth while still structured. Firm tannins are filled out with juicy red berry fruits and freshened with acidity. It's  already drinkable, although it will certainly be better from 2016.",
 'Towards the western end of the Douro vineyards, Avidagos is a fine place for table wines. This young, ripe and rounded wine is made to be drunk young. It has great black fruits and acidity that are linked by the firm tannin background. The wine will be ready from the end of 2016.',
 'Wood flavors dominate this young wine. It has structure and tannins, with concentrated toast and chocolate flavors. The extraction of the fruit gives the wine a bitter character. It should soften, so drink from 2019.',
 'Full of red berries and acidity, this accessible wine has a layer of tannins that does not detract from the general fruitiness. Drink from 2019.',
 "This relies on its ripe fruit to convey the rich flavors of the Douro. Its black fruit flavors dominate the bro

,points,vintage,designation_fe,winery_te,region_1_te,variety_te,province_te,country=Argentina,country=Armenia,country=Australia,...,1527,1528,1529,1530,1531,1532,1533,1534,1535,description
1,87,2011.0,0.000015,30.852118,27.113411,24.938141,29.612891,0,0,0,...,-0.019107,0.019660,-0.013772,0.004644,-0.004720,0.022807,0.020652,0.022175,0.004210,"This is ripe and fruity, a wine that is smooth..."
4840,87,2014.0,0.000015,30.289618,27.448935,25.095164,29.813062,0,0,0,...,-0.010552,0.000962,0.013695,0.020074,0.008086,-0.006263,0.028271,0.030030,-0.010298,Towards the western end of the Douro vineyards...
100505,88,2014.0,0.000439,31.817992,27.448935,25.095164,29.813062,0,0,0,...,-0.007045,0.029002,0.005638,0.003928,-0.017345,-0.011258,0.003505,0.040660,-0.021470,Wood flavors dominate this young wine. It has ...
22964,86,2014.0,0.000015,31.136291,27.079127,24.850589,28.716506,0,0,0,...,0.014728,-0.005501,0.006120,0.002407,0.009451,0.011342,0.018544,0.026511,-0.012374,"Full of red berries and acidity, this accessib..."
53335,87,2008.0,0.000008,32.411138,27.113411,24.938141,29.612891,0,0,0,...,-0.004752,0.014913,0.008214,0.011080,0.002991,0.005146,0.030613,0.024834,0.018017,This relies on its ripe fruit to convey the ri...


Odległości: [[0.0000000e+00 3.5559964e-07 5.7424052e-07 6.6156451e-07 8.3381877e-07]]
Indeksy: [[     1   4840 100505  22964  53335]]


In [5]:
# z wagami

df_nums = df.select_dtypes(include=['number'])
df_nums = df_nums.astype('float32') 
df_nums_new = prepare_weighted_features(df_nums, default_weight=4.0)
index, df_nums_array = build_faiss_index_from_df_nums(df_nums_new)
D, I = find_similar_items(index, df_nums_array[1], k=5)
wines_reviews_df, descriptions_list = get_wines_by_indices(I, df)

display(descriptions_list)
display(wines_reviews_df)

display(wines_reviews_df['price'])

print(f"Odległości: {D}")
print(f"Indeksy: {I}")

["This is ripe and fruity, a wine that is smooth while still structured. Firm tannins are filled out with juicy red berry fruits and freshened with acidity. It's  already drinkable, although it will certainly be better from 2016.",
 'Ripe and juicy, this is a full-bodied wine. It has some firm tannins under the red fruits and fresh acidity that gives a good structure. Wood aging has rounded out the mineral texture to give a rich feel at the end. It could age further, so drink from 2016.',
 'This is a fresh wine that is balanced between solid tannins and the ripe red berry fruits. Made to be drunk young, it has juicy acidity along with a firm mineral texture. Pure fruitiness shines at the end. Drink from late 2016.',
 "Firm and ripe, this is a richly juicy wine full of berries. Sweet tannins dominate the firmer structure at the back. There is some concentration and the wine is still young, so it's best to drink from 2016.",
 'Soft and fruity, this easy-drinking, ripe wine has soft tanni

,points,vintage,designation_fe,winery_te,region_1_te,variety_te,province_te,country=Argentina,country=Armenia,country=Australia,...,1527,1528,1529,1530,1531,1532,1533,1534,1535,description
1,87,2011.0,0.000015,30.852118,27.113411,24.938141,29.612891,0,0,0,...,-0.019107,0.019660,-0.013772,0.004644,-0.004720,0.022807,0.020652,0.022175,0.004210,"This is ripe and fruity, a wine that is smooth..."
30548,88,2012.0,0.000038,31.302824,27.113411,32.313847,19.903120,0,0,0,...,-0.003714,0.020401,-0.008879,0.005440,-0.013754,0.005435,0.019534,0.025517,-0.002265,"Ripe and juicy, this is a full-bodied wine. It..."
50449,86,2013.0,0.000008,32.469491,27.079127,24.850589,24.504628,0,0,0,...,0.004609,0.024847,0.007338,-0.009782,-0.009126,0.013596,0.016290,0.015523,0.002440,This is a fresh wine that is balanced between ...
28478,90,2009.0,0.000008,30.886158,27.483969,25.103417,27.941496,0,0,0,...,0.000755,0.032019,-0.019683,-0.006815,-0.008098,0.002066,0.027084,0.034256,-0.009567,"Firm and ripe, this is a richly juicy wine ful..."
40666,85,2013.0,0.000031,26.381695,27.113411,24.938141,21.337306,0,0,0,...,-0.010860,0.022758,-0.027804,0.000562,0.014849,0.003052,-0.001315,0.037537,-0.005495,"Soft and fruity, this easy-drinking, ripe wine..."


1        15.000000
30548     2.916083
50449    14.000000
28478    18.000000
40666    10.000000
Name: price, dtype: float64

Odległości: [[0.         0.02664061 0.0293086  0.02954409 0.03084605]]
Indeksy: [[    1 30548 50449 28478 40666]]


In [6]:
D, I = find_most_distant_items(index, df_nums_array[1], k=5)
wines_reviews_df, descriptions_list = get_wines_by_indices(I, df)

display(descriptions_list)
display(wines_reviews_df)

display(wines_reviews_df['price'])

print(f"Odległości: {D}")
print(f"Indeksy: {I}")


['Imported by JL Giguiere.',
 "This ripe wine shows plenty of blackberry fruits balanced well with some dry tannins. It is fresh, juicy with plenty of acidity, For a light vintage, it's perfumed, full of fresh flavors and will be ready to drink from 2017.",
 'Review not available at this time.',
 'Way too much banana, mango and sugar. Not a prime-time player.',
 'A superb wine from a great year, this is powerful and structured, with great acidity and solid, pronounced fruits. La Romanée is a small vineyard, wholly owned by Liger-Belair, next to Romanée-Conti. The wine is rich, spicy and very complex, with black fruits welling up from its depth. With great structure, it brings together opulent Pinot Noir fruits with firm, dense tannins with immense aging potential.']

,points,vintage,designation_fe,winery_te,region_1_te,variety_te,province_te,country=Argentina,country=Armenia,country=Australia,...,1527,1528,1529,1530,1531,1532,1533,1534,1535,description
33825,88,2007.0,0.288257,28.908926,27.113411,20.182311,21.827973,0,0,0,...,-0.009869,0.029841,-0.000315,-0.042521,-0.000084,0.019721,-0.025593,0.017463,-0.005591,Imported by JL Giguiere.
80290,88,2013.0,0.288257,35.363389,24.655247,46.318044,37.379250,0,0,0,...,-0.021824,0.030701,0.007642,-0.003681,0.006896,-0.009831,0.034058,0.014078,-0.020145,This ripe wine shows plenty of blackberry frui...
45752,84,2001.0,0.288257,31.708926,29.341413,32.124157,22.749602,0,0,0,...,0.004853,0.022757,0.001152,-0.033657,-0.001596,-0.018271,-0.009307,0.042485,-0.041028,Review not available at this time.
103991,80,2006.0,0.288257,22.159789,26.973298,34.698623,25.165822,1,0,0,...,0.007464,0.014252,-0.030805,0.008082,0.015676,-0.000884,0.020407,-0.006112,0.020652,"Way too much banana, mango and sugar. Not a pr..."
98380,96,2010.0,0.288257,292.971838,289.886158,47.531364,71.328735,0,0,0,...,-0.018937,0.026107,0.015188,-0.018041,-0.020529,0.012664,0.025093,0.009463,0.002997,"A superb wine from a great year, this is power..."


33825       16.0
80290     3300.0
45752       13.0
103991      10.0
98380     2500.0
Name: price, dtype: float64

Odległości: [[3.6847763 3.6876662 3.6881666 3.6891298 3.6895683]]
Indeksy: [[ 33825  80290  45752 103991  98380]]


In [7]:
df_vectors_only = df.select_dtypes(include=['number'])
indices = [1, 150, 12348]

display(df.iloc[indices]['description'].to_numpy())
D, I = recommend_from_multiple_wines([1, 150, 12348], df_vectors_only, index, 5)
wines_reviews_df, descriptions_list = get_wines_by_indices(I, df)

display(descriptions_list)
display(wines_reviews_df)

print(f"Odległości: {D}")
print(f"Indeksy: {I}")

array(["This is ripe and fruity, a wine that is smooth while still structured. Firm tannins are filled out with juicy red berry fruits and freshened with acidity. It's  already drinkable, although it will certainly be better from 2016.",
       'This wine shows off Yountville fruit at its finest, starting with a floral component to the nose. Spicy, bright and lifted, it coats the palate in ripe berry and cherry, ending with a snap of vanilla.',
       'With almost a quarter-million cases imported, this wine will provide a solid, medium-weight introduction to Marlborough Sauvignon Blanc. Herb and cut-grass notes accent the fig and melon fruit, backstopped by a lime-ridden finish.'],
      dtype=object)

["Aged in a combination of concrete egg, stainless steel and new French oak, this winning white is a study in peach and apricot—it's incredibly inviting in both taste and aroma. A layer of honey adds succulence without sweetness, as it remains dry and balanced through the mineral-laden finish.",
 "Owner Molly Chappellet's beloved baby, this white is deliciously crisp, dry and balanced, with layers of Meyer lemon, waxy apple and floral gardenia. Percolating acidity maintains its freshness beneath its rich texture and tropical succulence.",
 "This well-made wine is concise and focused, with pinpoint flavors of cut pear holding the core. Hints of thin red berry and a whiff of tea-tree oil animate the nose. It's unusual and well made.",
 "This is a white wine made from red Grenache and it's simultaneously intriguing and delicious, with sliced nectarine, kaffir lime, melon and bubblegum on the nose. Creamy flavors of poached pear made for a very rounded if texturally simple palate, which fi

,points,vintage,designation_fe,winery_te,region_1_te,variety_te,province_te,country=Argentina,country=Armenia,country=Australia,...,1527,1528,1529,1530,1531,1532,1533,1534,1535,description
9916,95,2016.0,0.000015,36.363519,62.318556,32.124157,38.991349,0,0,0,...,-0.016750,0.006317,0.000558,-0.002201,0.014998,-0.004344,-0.025743,0.013347,0.023610,"Aged in a combination of concrete egg, stainle..."
59865,92,2016.0,0.000654,55.165177,55.370662,23.949398,39.094196,0,0,0,...,-0.008530,0.034065,-0.007670,-0.009217,0.002256,0.006153,-0.018249,0.000674,-0.004212,"Owner Molly Chappellet's beloved baby, this wh..."
56424,91,2016.0,0.000031,40.096111,35.094659,22.976374,36.644986,0,0,0,...,0.008467,-0.001768,-0.005814,-0.000184,0.016459,0.005149,0.006061,0.023045,-0.025770,"This well-made wine is concise and focused, wi..."
89298,90,2016.0,0.000008,44.220661,47.798355,32.966717,38.991349,0,0,0,...,-0.004198,0.010001,0.001043,0.005996,-0.012985,0.031440,0.010429,0.039431,-0.000529,This is a white wine made from red Grenache an...
116840,94,2016.0,0.000008,32.350941,46.703552,26.482864,39.094196,0,0,0,...,0.001516,0.011221,-0.013166,-0.015338,-0.008303,-0.010142,0.004638,0.037378,0.027713,"With zero residual sugar, this winning white c..."


Odległości: [[1.6136676 1.6146185 1.6152713 1.6153764 1.6153864]]
Indeksy: [[  9916  59865  56424  89298 116840]]


## Embeding from user's wine description

In [8]:
def get_single_embedding(text, model, client):
    try:
        if not text:
            print("Text is null")
            return []
            
        text = text.replace("\n", " ")
        response = client.embeddings.create(input=[text], model=model)
        return response.data[0].embedding
        
    except Exception as e:
        print(f"Błąd przy generowaniu embeddingu: {e}")
        return []

In [9]:
some_wine_description = "A rich and full-bodied red wine with notes of blackberry, plum, and a hint of oak. Perfect for pairing with grilled meats or hearty stews."

import openai
from private import API_KEY

client = openai.OpenAI(api_key=API_KEY)
MODEL = "text-embedding-3-small"
embedding = get_single_embedding(some_wine_description, model=MODEL, client=client)

client_number = "001"
client_embedings_file_path = f"/clients_embeddings/embeddings_open_ai_api_for_client{client_number}.npy"
append_embedding_to_file(client_embedings_file_path, embedding)